# QuCoNot Module Guide

In this Jupyter notebook, we're going to demonstrate the multi-controlled Toffoli implementations, verifications and transformations based on the manuscript "Classification of permutation implementations for quantum computing".


## Implementations

We have implemented some of the multi-controlled Toffoli implementations listed in Table 1. First, we will start by importing each implementation.

<div>
<img src="img/summary_mct.png" width="800"/>
</div>

In [1]:
from quconot.implementations import (
    MCTBarenco74Dirty, 
    MCTBarenco75Dirty, 
    MCTNoAuxiliary, 
    MCTNoAuxiliaryRelative, 
    MCTNQubitDecomposition,
    MCTParallelDecomposition,
    MCTRecursion,
    MCTVChain,
    MCTVChainDirty
)

## Verifications

Next, we import the functions needed for verifying the implementations. Verification for each implementation is summarized in Table 2.

<div>
<img src="img/verification_table.png" width="800"/>
</div>

In [2]:
from quconot.verifications import (
    verify_circuit_strict_clean_non_wasting,
    verify_circuit_relative_clean_non_wasting,
    verify_circuit_strict_clean_wasting_entangled,
    verify_circuit_relative_clean_wasting_separable,
    verify_circuit_strict_clean_wasting_separable,
    verify_circuit_strict_dirty_non_wasting,
    verify_circuit_relative_dirty_non_wasting,
    verify_circuit_strict_dirty_wasting_entangled,
    verify_circuit_relative_dirty_wasting_separable,
    verify_circuit_strict_dirty_wasting_separable,
    verify_circuit_no_auxiliary,
    verify_circuit_no_auxiliary_relative
)

Using the imported functions, we create the necessary functions for printing the verification result.

In [68]:
from qiskit import Aer

usim = Aer.get_backend('unitary_simulator')

def print_result(classification, res):
    if res[0]:
        print("This implementation belongs to " + classification)
    else:
        print("This implementation doesn't belong to " + classification)
        
def verify_all(unitary, reverse_unitary, control_no, auxiliary_no):
    rd = {}
    
    rd["SCNW"] = verify_circuit_strict_clean_non_wasting(unitary, control_no, auxiliary_no)
    rd["RCNW"] = verify_circuit_relative_clean_non_wasting(unitary, control_no, auxiliary_no)
    rd["SCWE"] = verify_circuit_strict_clean_wasting_entangled(unitary, control_no, auxiliary_no)
    rd["RCWS"] = verify_circuit_relative_clean_wasting_separable(unitary, control_no, auxiliary_no)
    rd["SCWS"] = verify_circuit_strict_clean_wasting_separable(unitary, control_no, auxiliary_no)
    rd["SDNW"] = verify_circuit_strict_dirty_non_wasting(unitary, control_no, auxiliary_no)
    rd["RDNW"] = verify_circuit_relative_dirty_non_wasting(unitary, control_no, auxiliary_no)
    rd["SDWE"] = verify_circuit_strict_dirty_wasting_entangled(unitary, control_no, auxiliary_no)
    rd["RDWS"] = verify_circuit_relative_dirty_wasting_separable(reverse_unitary, control_no, auxiliary_no)
    rd["SDWS"] = verify_circuit_strict_dirty_wasting_separable(reverse_unitary, control_no, auxiliary_no)
    
    print_result("SDNW", rd["SDNW"])
    if rd["SDNW"][0]:
        print_result("SCNW", rd["SCNW"])
        print_result("RDNW", rd["RDNW"])
        print_result("SDWS", rd["SDWS"])

    if rd["RDNW"][0]:
        print_result("RCNW", rd["RCNW"])
        print_result("RDWS", rd["RDWS"])

    if rd["SDWS"][0]:
        print_result("SCWS", rd["SCWS"])
        print_result("RDWS", rd["RDWS"])

    if rd["SDWE"][0]:
        print_result("SCWE", rd["SCWE"])

    if rd["SCNW"][0]:
        print_result("RCNW", rd["RCNW"])
        print_result("SCWS", rd["SCWS"])

    if rd["SCWS"][0]:
        print_result("RCWS", rd["RCWS"])

    if rd["RCNW"][0]:
        print_result("RCWS", rd["RCWS"])

    if rd["RDWS"][0]:
        print_result("SDWE", rd["SDWE"])

    if rd["RCWS"][0]:
        print_result("SCWE", rd["SCWE"])


## Application

For a particular implementation, we will first generate the quantum circuit for MCT with 5 control qubits. Next, we will get the corresponding unitary. Finally, we will verify whether each implementation is correct using the imported functions. Note that the verifications should also follow the inclusion relations illustrated in Figure 1. 

<div>
<img src="img/classification.png" width="500"/>
</div>


### No Auxiliary

We will start with the most basic implementation which is No-Auxiliary.

In [69]:
control_no = 5
mct = MCTNoAuxiliary(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()

print_result("No Auxiliary", verify_circuit_no_auxiliary(unitary, control_no, mct.num_auxiliary_qubits()))
print_result("No Auxiliary Relative", verify_circuit_no_auxiliary_relative(unitary, control_no, mct.num_auxiliary_qubits()))

This implementation belongs to No Auxiliary
This implementation belongs to No Auxiliary Relative


In [70]:
control_no = 3
mct = MCTNoAuxiliaryRelative(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()

print_result("No Auxiliary", verify_circuit_no_auxiliary(unitary, control_no, mct.num_auxiliary_qubits()))
print_result("No Auxiliary Relative", verify_circuit_no_auxiliary_relative(unitary, control_no, mct.num_auxiliary_qubits()))

This implementation doesn't belong to No Auxiliary
This implementation belongs to No Auxiliary Relative


### Barenco

Now we have an example for Barenco 74, which is considered to be in the class "Relative Dirty Non-Wasting". Based on the above DAG, it should pass all the verifications associated with any class that is a superset of R-D-NW.

In [71]:
control_no = 5
mct = MCTBarenco74Dirty(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()
reverse_unitary = usim.run(circ.reverse_bits()).result().get_unitary()
auxiliary_no = mct.num_auxiliary_qubits()

verify_all(unitary, reverse_unitary, control_no, auxiliary_no)

# Examples of the expected to be failed

print("--- Failed ---")
print_result("SDNW", verify_circuit_strict_dirty_non_wasting(unitary, 5, mct.num_auxiliary_qubits()))
print_result("SCNW", verify_circuit_strict_clean_non_wasting(unitary, 5, mct.num_auxiliary_qubits()))

This implementation doesn't belong to DNW
This implementation belongs to CNWR
This implementation belongs to DWRS
This implementation belongs to CWRE
This implementation belongs to CWRS
This implementation belongs to DWRE
This implementation belongs to CWRE
--- Failed ---
This implementation doesn't belong to DNW
This implementation doesn't belong to CNW


As you can see from the result, the clean and dirty auxiliary tests are failed because they are outside of the DAG path.

Now, we have Barenco 75 which is an implementation without auxiliary:

In [72]:
mct = MCTBarenco75Dirty(5)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()

print_result("No Auxiliary", verify_circuit_no_auxiliary(unitary, control_no, mct.num_auxiliary_qubits()))
print_result("No Auxiliary Relative", verify_circuit_no_auxiliary_relative(unitary, control_no, mct.num_auxiliary_qubits()))

This implementation belongs to No Auxiliary
This implementation belongs to No Auxiliary Relative


### Qiskit

Next, we will consider implementations from Qiskit. In Qiskit, MCT can be implemented with parameters v-chain, v-chain-dirty, and recursion, which we wrapped inside the functions MCTVChain, MCTVChainDirty, and MCTRecursion. 

In [73]:
control_no = 5
mct = MCTVChain(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()
reverse_unitary = usim.run(circ.reverse_bits()).result().get_unitary()
auxiliary_no = mct.num_auxiliary_qubits()

verify_all(unitary, reverse_unitary, control_no, auxiliary_no)

This implementation doesn't belong to DNW
This implementation belongs to CWRE
This implementation belongs to CNWR
This implementation belongs to CWS
This implementation belongs to CWRS
This implementation belongs to CWRS
This implementation belongs to CWRE


In [74]:
control_no = 5
mct = MCTVChainDirty(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()
reverse_unitary = usim.run(circ.reverse_bits()).result().get_unitary()
auxiliary_no = mct.num_auxiliary_qubits()

verify_all(unitary, reverse_unitary, control_no, auxiliary_no)

This implementation belongs to DNW
This implementation belongs to CNW
This implementation belongs to DNWR
This implementation belongs to DWS
This implementation belongs to CNWR
This implementation belongs to DWRS
This implementation belongs to CWS
This implementation belongs to DWRS
This implementation belongs to CWRE
This implementation belongs to CNWR
This implementation belongs to CWS
This implementation belongs to CWRS
This implementation belongs to CWRS
This implementation belongs to DWRE
This implementation belongs to CWRE


Since V-Chain-Dirty belongs to "Strict Dirty Non-Wasting" class, it passes all the verifications.

In [75]:
control_no = 5
mct = MCTRecursion(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()
reverse_unitary = usim.run(circ.reverse_bits()).result().get_unitary()
auxiliary_no = mct.num_auxiliary_qubits()

verify_all(unitary, reverse_unitary, control_no, auxiliary_no)

This implementation belongs to DNW
This implementation belongs to CNW
This implementation belongs to DNWR
This implementation belongs to DWS
This implementation belongs to CNWR
This implementation belongs to DWRS
This implementation belongs to CWS
This implementation belongs to DWRS
This implementation belongs to CWRE
This implementation belongs to CNWR
This implementation belongs to CWS
This implementation belongs to CWRS
This implementation belongs to CWRS
This implementation belongs to DWRE
This implementation belongs to CWRE


Like V-Chain-Dirty, recursion belongs to "Strict Dirty Non-Wasting" class and it passes all the verifications.

### N-Qubit Decomposition

This is the implementation from paper ....


In [76]:
control_no = 5
mct = MCTNQubitDecomposition(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()
reverse_unitary = usim.run(circ.reverse_bits()).result().get_unitary()
auxiliary_no = mct.num_auxiliary_qubits()

verify_all(unitary, reverse_unitary, control_no, auxiliary_no)


This implementation belongs to DNW
This implementation belongs to CNW
This implementation belongs to DNWR
This implementation belongs to DWS
This implementation belongs to CNWR
This implementation belongs to DWRS
This implementation belongs to CWS
This implementation belongs to DWRS
This implementation belongs to CWRE
This implementation belongs to CNWR
This implementation belongs to CWS
This implementation belongs to CWRS
This implementation belongs to CWRS
This implementation belongs to DWRE
This implementation belongs to CWRE


### Parallel Decomposition

This is the implementation from paper ....

In [77]:
control_no = 5
mct = MCTParallelDecomposition(control_no)
circ = mct.generate_circuit()
unitary = usim.run(circ).result().get_unitary()
reverse_unitary = usim.run(circ.reverse_bits()).result().get_unitary()
auxiliary_no = mct.num_auxiliary_qubits()

verify_all(unitary, reverse_unitary, control_no, auxiliary_no)


This implementation doesn't belong to DNW
This implementation belongs to CWRE
This implementation belongs to CNWR
This implementation belongs to CWS
This implementation belongs to CWRS
This implementation belongs to CWRS
This implementation belongs to CWRE


# Transformations

In [ ]:
import time
from itertools import product

import numpy as np
from qiskit import QuantumCircuit, Aer, transpile

controls_to_check = 5


def lemma_7_2(num_controls=5):
    if num_controls < 3:
        raise ValueError("Number of controls must be >=3")

    n = 2 * num_controls - 1
    assert np.ceil(n / 2) == num_controls

    qc = QuantumCircuit(n)

    auxs = list(range(num_controls, n - 1))
    num_aux = len(auxs)

    controls = list(range(num_controls))
    target = n - 1
    print(n, controls, auxs, target)

    for i, c2 in enumerate(auxs[::-1]):
        qc.ccx(c2 - num_aux, c2, target - i)
    qc.ccx(0, 1, auxs[0])
    for i, c2 in enumerate(auxs):
        qc.ccx(c2 - num_aux, c2, target - num_aux + 1 + i)

    for i, c2 in enumerate(auxs[::-1]):
        if c2 == auxs[-1]:
            continue
        qc.ccx(c2 - num_aux, c2, target - i)
    qc.ccx(0, 1, auxs[0])
    for i, c2 in enumerate(auxs):
        if c2 == auxs[-1]:
            continue
        qc.ccx(c2 - num_aux, c2, target - num_aux + 1 + i)

    MCT = QuantumCircuit(n)
    MCT.mct(controls, target, auxs)

    return qc, MCT, controls, auxs, target

In [ ]:
simulator = Aer.get_backend('aer_simulator')
circ, mct, controls, auxs, target = lemma_7_2(controls_to_check)
print(circ.draw(fold=-1))
n = circ.num_qubits

bin_strs = [''.join(p) for p in product('10', repeat=n)]
inputs = [s[::-1] for s in bin_strs]
inputs = [list(s) for s in inputs]
inputs = np.asarray(inputs, dtype=int)

inputs

In [ ]:
def get_all_outputs(mct_circ):
    t1 = time.time()
    test_circs = [QuantumCircuit(n) for _ in bin_strs]
    for i, test_circ in enumerate(test_circs):
        test_circ.initialize(bin_strs[i])
        test_circ.append(mct_circ, range(n))
        test_circ.measure_all()
    test_circs = transpile(test_circs, simulator)
    res = simulator.run(test_circs, shots=100)
    counts = res.result().get_counts()
    outputs = [[s[::-1] for s in count.keys()] for count in counts]
    outputs = [list(s[0]) for s in outputs]
    outputs = np.asarray(outputs, dtype=int)
    print(time.time() - t1)
    return outputs


outputs_dirty = get_all_outputs(circ)
outputs_dirty

In [ ]:
def check_mct(all_inputs, all_outputs, all_controls, all_auxs, target, all_auxs_type_in='dirty',
              all_auxs_type_out='non_wasted'):
    num_all_controls = len(all_controls)
    # print(all_controls, all_auxs, target)
    if all_auxs_type_in == 'dirty' and all_auxs_type_out == 'non_wasted':
        same_in = (all_inputs[:, all_controls] == all_outputs[:, all_controls]).all()
        same_all_auxs = (all_inputs[:, all_auxs] == all_outputs[:, all_auxs]).all()
        check_active = np.where(np.sum(all_inputs[:, all_controls], axis=1) == num_all_controls)
        check_inactive = np.where(np.sum(all_inputs[:, all_controls], axis=1) != num_all_controls)
        active = (all_outputs[check_active, target] != all_inputs[check_active, target]).all()
        inactive = (all_outputs[check_inactive, target] == all_inputs[check_inactive, target]).all()
        return all([same_in, same_all_auxs, active, inactive])
    if all_auxs_type_in == 'dirty' and all_auxs_type_out == 'wasted':
        same_in = (all_inputs[:, all_controls] == all_outputs[:, all_controls]).all()
        # same_all_auxs = (all_inputs[:, all_auxs] == all_outputs[:, all_auxs]).all()
        check_active = np.where(np.sum(all_inputs[:, all_controls], axis=1) == num_all_controls)
        check_inactive = np.where(np.sum(all_inputs[:, all_controls], axis=1) != num_all_controls)
        active = (all_outputs[check_active, target] != all_inputs[check_active, target]).all()
        inactive = (all_outputs[check_inactive, target] == all_inputs[check_inactive, target]).all()
        return all([same_in, True, active, inactive])
    if all_auxs_type_in == 'clean' and all_auxs_type_out == 'non_wasted':
        clean_idx = np.where(np.sum(all_inputs[:, all_auxs], axis=1) == 0)
        all_inputs = all_inputs[clean_idx]
        all_outputs = all_outputs[clean_idx]
        # print(clean_idx, all_inputs, all_outputs)
        same_in = (all_inputs[:, all_controls] == all_outputs[:, all_controls]).all()
        same_all_auxs = (all_inputs[:, all_auxs] == all_outputs[:, all_auxs]).all()
        check_active = np.where(np.sum(all_inputs[:, all_controls], axis=1) == num_all_controls)
        check_inactive = np.where(np.sum(all_inputs[:, all_controls], axis=1) != num_all_controls)
        active = (all_outputs[check_active, target] != all_inputs[check_active, target]).all()
        inactive = (all_outputs[check_inactive, target] == all_inputs[check_inactive, target]).all()
        return all([same_in, same_all_auxs, active, inactive])
    if all_auxs_type_in == 'clean' and all_auxs_type_out == 'wasted':
        clean_idx = np.where(np.sum(all_inputs[:, all_auxs], axis=1) == 0)
        all_inputs = all_inputs[clean_idx]
        all_outputs = all_outputs[clean_idx]
        # print(clean_idx, all_inputs, all_outputs)
        same_in = (all_inputs[:, all_controls] == all_outputs[:, all_controls]).all()
        # same_all_auxs = (all_inputs[:, all_auxs] == all_outputs[:, all_auxs]).all()
        check_active = np.where(np.sum(all_inputs[:, all_controls], axis=1) == num_all_controls)
        check_inactive = np.where(np.sum(all_inputs[:, all_controls], axis=1) != num_all_controls)
        active = (all_outputs[check_active, target] != all_inputs[check_active, target]).all()
        inactive = (all_outputs[check_inactive, target] == all_inputs[check_inactive, target]).all()
        return all([same_in, True, active, inactive])


def transform_to_clean(circuit, all_controls, all_auxs, target):
    all_auxs = all_auxs.copy()
    num_all_controls = len(all_controls)
    num_all_auxs = len(all_auxs)

    instructions = circuit.data
    remove_ins = []
    for i, instruction in enumerate(instructions):
        c1 = instruction.qubits[0]._index
        c2 = instruction.qubits[1]._index
        t = instruction.qubits[2]._index
        if c1 in all_auxs:
            remove_ins.append(i)
            continue
        if c2 in all_auxs:
            remove_ins.append(i)
            continue
        if t in all_auxs:
            all_auxs.remove(t)
            continue

    remaining_instructions = [i for j, i in enumerate(instructions) if j not in set(remove_ins)]

    clean_circuit = QuantumCircuit(num_all_controls + num_all_auxs + 1)
    for instruction in remaining_instructions:
        if instruction.operation.name == 'ccx':
            c1 = instruction.qubits[0]._index
            c2 = instruction.qubits[1]._index
            t = instruction.qubits[2]._index
            clean_circuit.ccx(c1, c2, t)

    return clean_circuit


def transform_to_wasted(circuit, all_controls, all_auxs, target):
    all_auxs = all_auxs.copy()
    num_all_controls = len(all_controls)
    num_all_auxs = len(all_auxs)

    instructions = circuit.data
    remove_ins = []
    for i, instruction in enumerate(instructions[::-1]):
        c1 = instruction.qubits[0]._index
        c2 = instruction.qubits[1]._index
        t = instruction.qubits[2]._index
        if t in all_auxs:
            remove_ins.append(i)
            continue
        else:
            break

    remaining_instructions = [i for j, i in enumerate(instructions[::-1]) if j not in set(remove_ins)]

    wasted_circuit = QuantumCircuit(num_all_controls + num_all_auxs + 1)
    for instruction in remaining_instructions[::-1]:
        if instruction.operation.name == 'ccx':
            c1 = instruction.qubits[0]._index
            c2 = instruction.qubits[1]._index
            t = instruction.qubits[2]._index
            wasted_circuit.ccx(c1, c2, t)

    return wasted_circuit


In [ ]:
clean_circ = transform_to_clean(circ, controls, auxs, target)
print(clean_circ.draw(fold=-1))
outputs_clean = get_all_outputs(clean_circ)

print(outputs_clean)


In [ ]:
wasted_circ = transform_to_wasted(circ, controls, auxs, target)
print(wasted_circ.draw(fold=-1))
outputs_wasted = get_all_outputs(wasted_circ)

print(outputs_wasted)

In [ ]:
clean_wasted_circ = transform_to_wasted(clean_circ, controls, auxs, target)
print(clean_wasted_circ.draw(fold=-1))
outputs_clean_wasted = get_all_outputs(clean_wasted_circ)

print(outputs_clean_wasted)

In [ ]:
print(check_mct(inputs, outputs_dirty, controls, auxs, target, all_auxs_type_in='dirty', all_auxs_type_out='non_wasted'))
print(
    check_mct(inputs, outputs_clean, controls, auxs, target, all_auxs_type_in='clean', all_auxs_type_out='non_wasted'))
print(check_mct(inputs, outputs_wasted, controls, auxs, target, all_auxs_type_in='dirty', all_auxs_type_out='wasted'))
print(check_mct(inputs, outputs_clean_wasted, controls, auxs, target, all_auxs_type_in='clean',
                all_auxs_type_out='wasted'))
